In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [1]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q1. Notebook 

What's the standard deviation of the predicted duration for this dataset?

In [6]:
import numpy as np

std_pred = np.std(y_pred)
print(f"Standard deviation of predicted durations: {std_pred:.2f}")

Standard deviation of predicted durations: 6.25


### Q2. Preparing the output

In [14]:
year = 2023
month = 3

In [15]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [16]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [17]:
output_file = 'predictions_2023_03.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

What's the size of the output file?

In [18]:
import os 

file_size_mb = os.path.getsize(output_file) / (1024 * 1024)

print(f"File size: {os.path.getsize(output_file)} bytes ({file_size_mb:.2f} MB)")

File size: 68641024 bytes (65.46 MB)


### 3. Creating the scoring script

``` bash jupyter nbconvert --to script starter.ipynb ```

``` bash mv starter.py homework.py ```

### Q4. Virtual environment

What's the first hash for the Scikit-Learn dependency?

sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c

### Q5. Parametrize the script

python homework.py --year 2023 --month 4

Mean predicted duration: 14.29 minutes

Standard deviation of predicted duration: 6.35 minutes

### Q6. Docker container 

First we create a Docker file 

FROM agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

WORKDIR /app

COPY homework.py .

RUN pip install pandas pyarrow numpy scikit-learn==1.5.0

Then we do docker build -t duration-predictor-v2 . 

Then we do docker run --rm -v ${PWD}:/app duration-predictor-v2 python homework.py --year 2023 --month 5

Mean predicted duration: 14.24 minutes

Standard deviation of predicted duration: 6.26 minutes